In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 15, 6
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
%run period.py 64 3 0.25 --model_type gru --sigma 1e-9 --sim_type period_only/even/noise0 --even
pred_gru = model.predict(X[test])

[]
/Users/brettnaul/Dropbox/Documents/timeflow/keras_logs/period_only/even/noise0/gru_064_x3_2m03_drop25
Loading /Users/brettnaul/Dropbox/Documents/timeflow/keras_logs/period_only/even/noise0/gru_064_x3_2m03_drop25/weights.h5...


In [15]:
[model.predict(X[0:5])[:, 0], Y[0:5, 0]]

[array([ 1.6712625 ,  2.47691607,  2.57400465,  2.51226425,  2.63632059], dtype=float32),
 array([ 1.68369399,  1.34468168,  1.55645163,  1.69378167,  2.07775232])]

In [ ]:
%%time
from gatspy.periodic import LombScargle, LombScargleFast

pred_gatspy = np.zeros(pred_gru.shape)
for i in range(args.N_test):
#    opt_args = {'period_range': (0.1, 0.9 * (X[test[i]][-1, 0] - X[test[i]][0, 0])), 'quiet': True}
    opt_args = {'period_range': (0.05, 2 * np.pi), 'quiet': True}
    model_gatspy = LombScargle(fit_period=True, optimizer_kwds=opt_args)#, silence_warnings=True)
    model_gatspy.fit(np.linspace(0, 2 * np.pi, args.n_max), X[test[i]][:, 0])
    omega = 2 * np.pi / model_gatspy.best_period
    off, A1, A2 = model_gatspy._best_params(omega)
    pred_gatspy[i] = np.array([omega / 2 / np.pi, np.sqrt(A1 ** 2 + A2 ** 2), np.arctan2(A2, A1), off + model_gatspy.ymean_])

In [ ]:
i = -1

In [ ]:
i += 1

t = np.linspace(0, 2 * np.pi, args.n_max)
x = X[test[i]].ravel()
plt.plot(t, x, 'o')

w, A, phi, b = Y[test[i]]
plt.plot(t, A * np.sin(2 * np.pi * w * t + phi) + b, '-')

w_c, A_c, phi_c, b_c = pred_conv[i]
x_c = A_c * np.sin(2 * np.pi * w_c * t + phi_c) + b_c
plt.plot(t, x_c, '--')

w_r, A_r, phi_r, b_r = pred_gru[i]
x_r = A_r * np.sin(2 * np.pi * w_r * t + phi_r) + b_r
plt.plot(t, x_r, '--')

#omega = 2 * np.pi / model_gatspy.best_period
#off, A1, A2 = model_gatspy._best_params(omega)
#w_g, A_g, phi_g, b_g = [omega / 2 / np.pi, np.sqrt(A1 ** 2 + A2 ** 2), np.arctan2(A2, A1), off + model_gatspy.ymean_]
w_g, A_g, phi_g, b_g = pred_gatspy[i]
x_g = A_g * np.sin(2 * np.pi * w_g * t + phi_g) + b_g
#x_g = model_gatspy.fit(t, x).predict(t)
plt.plot(t, x_g, '--')

plt.legend(['Noisy', 'Original', 'Convolutional', 'GRU', 'Gatspy'])
plt.title("True ({:3f}, {:3f}, {:3f}, {:3f}), Conv ({:3f}, {:3f}, {:3f}, {:3f}), GRU ({:3f}, {:3f}, {:3f}, {:3f}), Gatspy ({:3f}, {:3f}, {:3f}, {:3f})".format(
    w, A, phi, b,
    w_c, A_c, phi_c, b_c,
    w_r, A_r, phi_r, b_r,
    w_g, A_g, phi_g, b_g,
    ))

In [ ]:
sns.jointplot(pred_conv[:, 0] - Y[test, 0], pred_gatspy[:, 0] - Y[test, 0])